## 告别黑盒！用 LangSmith 打造大模型调用链追踪与观测工具

大模型回答问题不准确，难以调试？

- **盲目猜测**：是 Prompt 有问题？还是检索召回不准？
- **日志地狱**：翻遍各种日志文件，拼凑执行轨迹
- **重现困难**：线上问题在开发环境死活复现不了
- **成本失控**：不知道哪些调用在疯狂烧 Token

传统的 APM 工具（如 Datadog、New Relic）只能告诉你"API 调用失败了"，但对于大模型应用的内部逻辑——Prompt 构建、向量检索、多轮对话状态管理等，完全是盲区。

"开箱即用"的大模型可观测性方案，让你的 LangChain 应用告别黑箱时代。


## 传统监控工具为什么不行？

- 粒度太粗：传统工具只能监控到 HTTP 请求级别
- 上下文缺失：LLM 应用往往涉及复杂的数据流转，传统工具无法追踪这种语义级的执行链路。
- 成本盲区：Token 消耗是大模型应用的核心成本，但传统监控完全感知不到哪个环节在"烧钱"。

## LangSmith 的设计哲学：分层埋点 + 上下文传播

**核心概念解析**

Trace（调用链）：一次完整的用户请求执行过程，比如"用户问问题→Agent回答"这整个流程。

Run（运行步骤）：Trace 内部的每个执行单元，可以是一次 LLM 调用、一次向量检索、一次 Prompt 渲染等。

追踪树结构：Run 之间形成父子关系，构成可无限展开的层级结构，完美映射 LangChain 表达式语言（LCEL）的执行图。


## 技术实现原理

LangSmith 的追踪机制基于**上下文传播**技术：

```python
# LangSmith 自动注入的追踪逻辑
def trace_run(func, run_type, inputs):
    run_id = generate_run_id()
    parent_run_id = get_current_run_context()
    
    # 记录开始时间和输入
    start_time = time.time()
    log_run_start(run_id, parent_run_id, run_type, inputs)
    
    try:
        # 执行实际逻辑
        with run_context(run_id):
            outputs = func(inputs)
        
        # 记录成功结果
        log_run_success(run_id, outputs, time.time() - start_time)
        return outputs
    
    except Exception as e:
        # 记录错误信息
        log_run_error(run_id, str(e), time.time() - start_time)
        raise
```

### 与 LCEL 执行图的同构映射

比如这样一个 LangChain 流程：

```python
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
```
在 LangSmith 中会生成对应的追踪树：

```raw
📊 Trace: 用户问答会话
├── 🔍 Run: 向量检索 (retriever)
│   ├── 输入: "我的订单什么时候发货？"
│   ├── 输出: [相关文档片段]
│   └── 耗时: 120ms, Token: 0
├── 📝 Run: Prompt 构建 (prompt)
│   ├── 输入: {context: [...], question: "..."}
│   ├── 输出: "根据以下信息回答用户问题..."
│   └── 耗时: 5ms, Token: 0
├── 🤖 Run: LLM 调用 (llm)
│   ├── 输入: "根据以下信息回答用户问题..."
│   ├── 输出: "您的订单预计在3-5个工作日内发货"
│   └── 耗时: 800ms, Token: 150
└── ✂️ Run: 输出解析 (StrOutputParser)
    ├── 输入: "您的订单预计在3-5个工作日内发货"
    ├── 输出: "您的订单预计在3-5个工作日内发货"
    └── 耗时: 1ms, Token: 0
```


### 实际价值体现

通过这种追踪树架构，我们能够：

精确定位问题：当用户反馈回答不准确时，可以直接看到是检索环节召回了错误文档，还是 Prompt 构建有问题。

优化性能瓶颈：清楚看到每个步骤的耗时分布，比如发现向量检索占了总时间的 60%，就知道该优化索引了。

控制成本支出：Token 消耗精确到每个 LLM 调用，能够识别出哪些 Prompt 过于冗长，哪些调用可以缓存。

复现线上问题：完整的输入输出记录让问题复现变得轻而易举，不再需要"盲人摸象"式的调试。

## 实战部署

### 第一步：注册账号获取 API 密钥

访问 https://smith.langchain.com/ 注册账号。

注册完成后，进入 Settings → API Keys，点击 "Create API Key"


### 第二步：配置关键环境变量

```python
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "pr-wilted-temporary-9"
# os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"
```

### 第三步：安装软件包

```python
pip install -U langgraph "langchain[openai]"
```

### 第四步：零代码修改，直接运行

In [ ]:

import os


from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"


agent = create_react_agent(
    model="openai:gpt-5-mini",
    tools=[get_weather],
    prompt="You are a helpful assistant.",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "What is the weather in San Francisco?"}]}
)

### 在 LangSmith 控制台查看追踪结果

- 性能监控：关键指标深度解读
- 核心监控指标全景图
- 延迟分析：性能瓶颈的透视镜
- Token 使用分析：成本控制的利器
- 错误监控：故障预警系统
- 用户行为分析：业务价值挖掘

### 其他注意事项

个人版每月 5000 条追踪记录的限制